In [1]:
import numpy as np
import pandas as pd

In [2]:
#data = np.random.random((3,3))
data = pd.read_csv('https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv')
data = data.select_dtypes(include='number').values
#cov = np.dot(data.T, data)

In [3]:
def sort_pairs(eigenvalues, eigenvectors):
    eigenpairs = [[0,[]] for _ in range(len(eigenvalues))]
    for i in range(len(eigenvalues)):
        if eigenvalues[i] < 0:
            eigenpairs[i][0] = -1*eigenvalues[i]
            eigenpairs[i][1] = -1*eigenvectors[i]
        else:
            eigenpairs[i][0] = eigenvalues[i]
            eigenpairs[i][1] = eigenvectors[i]
    eigenpairs.sort(key=lambda eigpair: eigpair[0], reverse=True)
    eigenvalues, eigenvectors = [], []
    for eigenpair in eigenpairs:
        eigenvalues.append(eigenpair[0])
        eigenvectors.append(np.array(eigenpair[1]))
    return np.array(eigenvalues), np.array(eigenvectors)
#do poprawki
#def sort_pairs(eigenvalues, eigenvectors):
    #return eigenvalues, eigenvectors

<h2>numpy - Eigen decomposition</h2>

In [4]:
def check_symmetric(a, tol=1e-8):
    return np.allclose(a, a.T, atol=tol)

def eigen_decomposition(A):
    if check_symmetric(A):
        eigenvalues, eigenvectors = np.linalg.eigh(A)
    else: 
        eigenvalues, eigenvectors = np.linalg.eig(A)
    return eigenvalues, eigenvectors

<h2>numpy - SVD</h2>

In [86]:
def svd_decomposition(A, gen_HHtransposed=False): #do usuniecia debug
    u,s,v = np.linalg.svd(A)
    s*=s
    if gen_HHtransposed:
        zeros = np.zeros((u.shape[0]-len(s),))
        print(zeros.shape, s.shape)
        s = np.concatenate((s, zeros))
        return s, u
    else:
        return s, v.T
    
#sprawdzić czy poprawne zwracanie u/v

<h2>numpy - QR</h2>

In [83]:
def qr_decomposition(A):
    X = A
    D = np.diag([1 for _ in range(A.shape[1])])
    #i = 0
    while not np.allclose(X, np.triu(X)):
        Q, R = np.linalg.qr(X)
        D = np.dot(D, Q)
        X = np.dot(R, Q)
        #i += 1
    #print(i)
    return np.diagonal(X), D

# probelmy z wydajnością. przeanalizować https://link-1springer-1com-1htv89mdl09c7.hansolo.bg.ug.edu.pl/article/10.1007/s13042-012-0131-7

<h2>Testy poprawności</h2>

<h3>H<sup>T</sup>H</h3>

In [43]:
cov = np.dot(data.T, data)
cov.shape

(4, 4)

<h4>Eigendecomposition</h4>

In [44]:
evalsE, evecsE = eigen_decomposition(cov)

<h4>QR decomposition</h4>

In [45]:
evalsQ, evecsQ = qr_decomposition(cov)

<h4>Singular value decomposition</h4>

In [54]:
evalsS, evecsS = svd_decomposition(data)

<h5>Eigenvalues MSE</h5>

In [55]:
((np.array(sorted(evalsE)) - np.array(sorted(evalsQ)))**2).mean(axis=None)

1.8496721411861048e-24

In [56]:
((np.array(sorted(evalsE)) - np.array(sorted(evalsS)))**2).mean(axis=None)

8.42528887540234e-25

In [57]:
((np.array(sorted(evalsQ)) - np.array(sorted(evalsS)))**2).mean(axis=None)

8.584709775738454e-25

<h5>Decomp MSE</h5>

In [58]:
((evecsE.dot(np.diag(evalsE)).dot(np.linalg.inv(evecsE)) - cov)**2).mean(axis=None)

9.208846663188003e-25

In [59]:
((evecsS.dot(np.diag(evalsS)).dot(np.linalg.inv(evecsS)) - cov)**2).mean(axis=None)

3.1083896456094243e-24

In [61]:
((evecsQ.dot(np.diag(evalsQ)).dot(np.linalg.inv(evecsQ)) - cov)**2).mean(axis=None)

5.118613138086932e-18

<h3>HH<sup>T</sup></h3>

In [62]:
cov = np.dot(data, data.T)
cov.shape

(150, 150)

<h4>Eigendecomposition</h4>

In [76]:
evalsE, evecsE = eigen_decomposition(cov)
evalsE.shape, evecsE.shape

((150,), (150, 150))

<h4>QR decomposition</h4>

In [80]:
evalsQ, evecsQ = qr_decomposition(cov)

<h4>Singular value decomposition</h4>

In [87]:
evalsS, evecsS = svd_decomposition(data, True)

(146,) (4,)


<h5>Eigenvalues MSE</h5>

In [96]:
((np.array(sorted(abs(evalsE))) - np.array(sorted(abs(evalsQ))))**2).mean(axis=None)

2.1358755921609817e-24

In [97]:
((np.array(sorted(abs(evalsE))) - np.array(sorted(abs(evalsS))))**2).mean(axis=None) #problem. SVD generuje S tylko dla wartości > 0.

2.4302324110042573e-24

In [98]:
((np.array(sorted(abs(evalsQ))) - np.array(sorted(abs(evalsS))))**2).mean(axis=None) #problem. SVD generuje S tylko dla wartości > 0.

9.075115810946327e-26

<h5>Decomp MSE</h5>

In [99]:
((evecsE.dot(np.diag(evalsE)).dot(np.linalg.inv(evecsE)) - cov)**2).mean(axis=None)

4.4593834356810356e-26

In [101]:
((evecsS.dot(np.diag(evalsS)).dot(np.linalg.inv(evecsS)) - cov)**2).mean(axis=None)

5.0755274739983593e-26

In [100]:
((evecsQ.dot(np.diag(evalsQ)).dot(np.linalg.inv(evecsQ)) - cov)**2).mean(axis=None)

4.799506414789542e-21

<h2>Wydajność</h2>

<h4>m > n</h4>

In [ ]:
A = np.random.random((10,3))

In [ ]:
%%timeit -n 100
cov = np.dot(A.T, A)
eigen_decomposition(cov)

In [ ]:
%%timeit -n 100
svd_decomposition(A)

In [ ]:
A = np.random.random((100,4))

In [ ]:
%%timeit -n 100
cov = np.dot(A.T, A)
eigen_decomposition(cov)

In [ ]:
%%timeit -n 100
svd_decomposition(A)

In [ ]:
A = np.random.random((500,20))

In [ ]:
%%timeit -n 100
cov = np.dot(A.T, A)
eigen_decomposition(cov)

In [ ]:
%%timeit -n 100
svd_decomposition(A)

<h4>m < n</h4>

In [ ]:
A = np.random.random((3,10))

In [ ]:
%%timeit -n 100
cov = np.dot(A.T, A)
eigen_decomposition(cov)

In [ ]:
%%timeit -n 100
svd_decomposition(A)

In [ ]:
A = np.random.random((4,100))

In [ ]:
%%timeit -n 100
cov = np.dot(A.T, A)
eigen_decomposition(cov)

In [ ]:
%%timeit -n 100
svd_decomposition(A)

In [ ]:
A = np.random.random((20,500))

In [ ]:
%%timeit -n 100
cov = np.dot(A.T, A)
eigen_decomposition(cov)

In [ ]:
%%timeit -n 100
svd_decomposition(A)